In [2]:
from langchain import PromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import CTransformers
from langchain.chains import RetrievalQA
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

In [3]:
def load_llm():
    llm = CTransformers(
        model='llama-2-7b-chat.ggmlv3.q8_0.bin',
        model_type='llama',
        max_new_token=512,
        temperature=0.5
    )
    return llm


In [4]:
def retrieval_qa_chain(llm, prompt, db: FAISS):
    qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                           chain_type='stuff',
                                           retriever=db.as_retriever(
                                               search_kwargs={'k': 3}),
                                           return_source_documents=True,
                                           chain_type_kwargs={"verbose": True, "prompt": prompt,
                                                              }
                                           )
    return qa_chain


In [4]:
def set_custom_conversational_prompts():
    condense_prompt = PromptTemplate.from_template(
        ('Do X with user input ({question}), and do Y with chat history ({chat_history}).')
    )

    combine_docs_custom_prompt = PromptTemplate.from_template(
        ('Write a haiku about a dolphin.\n\n'
         'Completely ignore any context, such as {context}, or the question ({question}).')
    )
    return condense_prompt, combine_docs_custom_prompt

In [5]:

def conversational_retrieval_chain(llm, db: FAISS):
    condense_prompt, combine_docs_custom_prompt = set_custom_conversational_prompts()

    memory = ConversationBufferMemory(
        memory_key="chat_history",
        return_messages=True,)

    chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=db.as_retriever(search_kwargs={'k': 2}),
        memory=memory,
        condense_question_prompt=condense_prompt,
        verbose=True,
        combine_docs_chain_kwargs=dict(prompt=combine_docs_custom_prompt)
    )
    return chain

In [6]:
DB_PATH = "./vectorstores/db_faiss"
def qa_bot():
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2", model_kwargs={'device': 'cpu'})

    db = FAISS.load_local(folder_path=DB_PATH, embeddings=embeddings)

    llm = load_llm()

    qa = conversational_retrieval_chain(llm=llm,db=db)
    return qa

This cell contais an example with an alterated condense and combine docs prompts that make the model ignore the chat history and context completely, instead it is forced to make a japanese poem about dolphins.

In [7]:

print(qa_bot()({'question': "What is the treatment for diabetes?"}))

c:\Users\torre\.virtualenvs\llama-medical-bot-or6BXnwr\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\torre\.virtualenvs\llama-medical-bot-or6BXnwr\Lib\site-packages\bitsandbytes\cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a haiku about a dolphin.

Completely ignore any context, such as Nancy Ross-Flanigan
Antidiabetic drugs
Definition
Antidiabetic drugs are medicines that help control
blood sugar levels in people with diabetes mellitus
(sugar diabetes).
Purpose
Diabetes may be divided into type I and type II, for-
merly termed juvenile onset or insulin-dependent, and
GALE ENCYCLOPEDIA OF MEDICINE 2 261Antidiabetic drugsGEM - 0001 to 0432 - A  10/22/03 1:42 PM  Page 261

andpermit glucose control over longer periods of time., or the question (What is the treatment for diabetes?).

> Finished chain.

> Finished chain.
{'question': 'What is the treatment for diabetes?', 'chat_history': [HumanMessage(content='What is the treatment for diabetes?', additional_kwargs={}, example=False), AIMessage(content=" Instead, focus on a specific aspect o

Now we will work with coherent prompts

In [7]:
def set_custom_conversational_prompts():
    condense_prompt = PromptTemplate.from_template(
        ('Answer the question ({question}), use the chat history ({chat_history}) to infer valuable information to formulate a complete answer.')
    )

    combine_docs_custom_prompt = PromptTemplate.from_template(
        ("""You are a helpful, medical assistant.If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. Use the following pieces of information to answer the user's question.
        
        Context: {context}
        
        Question: {question}
Only return the helpful answer below and nothing else.Helpful answer:
        """)
    )
    return condense_prompt, combine_docs_custom_prompt

In [8]:
print(qa_bot()({'question': "What is the treatment for diabetes?"}))


c:\Users\torre\.virtualenvs\llama-medical-bot-or6BXnwr\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\torre\.virtualenvs\llama-medical-bot-or6BXnwr\Lib\site-packages\bitsandbytes\cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are a helpful, medical assistant.If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. Use the following pieces of information to answer the user's question.
        
        Context: Nancy Ross-Flanigan
Antidiabetic drugs
Definition
Antidiabetic drugs are medicines that help control
blood sugar levels in people with diabetes mellitus
(sugar diabetes).
Purpose
Diabetes may be divided into type I and type II, for-
merly termed juvenile onset or insulin-dependent, and
GALE ENCYCLOPEDIA OF MEDICINE 2 261Antidiabetic drugsGEM - 0001 to 0432 - A  10/22/03 1:42 PM  Page 261

andpermit glucose control over longer periods of time.
        
        Question: What is the treatment

In [8]:
def llama_v2_prompt(
    messages: list[dict]
):
    B_INST, E_INST = "[INST]", "[/INST]"
    B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
    BOS, EOS = "<s>", "</s>"
    DEFAULT_SYSTEM_PROMPT = """You are a helpful, respectful and honest medical assistant.Use the following pieces of information to answer the user's question.If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
    Use the following pieces of information to answer the user's question.

    Context: {context}
    Question: {question}

    Only return the helpful answer below and nothing else.
    Helpful answer:
    """

    if messages[0]["role"] != "system":
        messages = [
            {
                "role": "system",
                "content": DEFAULT_SYSTEM_PROMPT,
            }
        ] + messages
    messages = [
        {
            "role": messages[1]["role"],
            "content": B_SYS + messages[0]["content"] + E_SYS + messages[1]["content"],
        }
    ] + messages[2:]

    messages_list = [
        f"{BOS}{B_INST} {(prompt['content']).strip()} {E_INST} {(answer['content']).strip()} {EOS}"
        for prompt, answer in zip(messages[::2], messages[1::2])
    ]
    messages_list.append(
        f"{BOS}{B_INST} {(messages[-1]['content']).strip()} {E_INST}")

    return "".join(messages_list)


message = llama_v2_prompt(
    [
        {'role': 'user', 'content': 'User_Msg_1'},
        {'role': 'assistant', 'content': 'Asst_Msg_1'},
        {'role': 'user', 'content': 'User_Msg_2'},
        {'role': 'assistant', 'content': 'Asst_Msg_2'},
        {'role': 'user', 'content': 'User_Msg_3'}]
)
print(message)

<s>[INST] <<SYS>>
You are a helpful, respectful and honest medical assistant.Use the following pieces of information to answer the user's question.If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
    Use the following pieces of information to answer the user's question.

    Context: {context}
    Question: {question}

    Only return the helpful answer below and nothing else.
    Helpful answer:
    
<</SYS>>

User_Msg_1 [/INST] Asst_Msg_1 </s><s>[INST] User_Msg_2 [/INST] Asst_Msg_2 </s><s>[INST] User_Msg_3 [/INST]
